# RRAP-IS Authentication Demo Notebook

> A tutorial of RRAP authentication using Jupyter notebooks.

- toc: true 
- badges: true
- comments: true
- categories: [jupyter, authentication]

## About

This notebook is a demonstration of authenticating with the RRAP-IS system.  

## Run all imports

Keep all your imports at the top of a notebook.  It allows for easier management.

In [1]:
import requests
import os
import sys
import json
from IPython.core.display import display, HTML
from IPython.display import IFrame
from mdsisclienttools.auth.TokenManager import DeviceFlowManager

/tmp/ipykernel_15603/723406562.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Define global variables

Similar to import we like to define notebook variable at the top and reuse them throughout the notebook

In [2]:
data_api = "https://data-api.testing.rrap-is.com"
registry_api = "https://registry-api.testing.rrap-is.com"
prov_api = "https://prov-api.testing.rrap-is.com"
auth_server = "https://auth.dev.rrap-is.com/auth/realms/rrap"
# garbage = "https://frogs.are.green"
base_urls = {'data_api': data_api, 'registry_api': registry_api, 'prov_api': prov_api, 'auth_server': auth_server}#, 'garbage': garbage}
display(f'Checking base urls')

for key, url in base_urls.items():
    try:
        print(f'Testing - {url}', end="")
        r = requests.get(url)
        r.raise_for_status()
        print(f' - Passed')
    except requests.exceptions.HTTPError as err:
        print(f' - Fail')
        raise SystemExit(err)
    except requests.exceptions.RequestException as e:
        # catastrophic error. bail.
        print(f' - Fail')
        raise SystemExit(e)

'Checking base urls'

Testing - https://data-api.testing.rrap-is.com - Passed
Testing - https://registry-api.testing.rrap-is.com - Passed
Testing - https://prov-api.testing.rrap-is.com - Passed
Testing - https://auth.dev.rrap-is.com/auth/realms/rrap - Passed


## Authentication steps

### Setup tokens using device authorisation flow against keycloak server

This could result in a browser window being opened if you don't have valid tokens cached in local storage.

In [3]:
# this caches the tokens
local_token_storage = ".tokens.json"

token_manager = DeviceFlowManager(
    stage="TEST",
    keycloak_endpoint=auth_server,
    local_storage_location=local_token_storage
)

Attempting to generate authorisation tokens.

Looking for existing tokens in local storage.

Validating found tokens

Trying to use found tokens to refresh the access token.

Tokens found in storage but they are not valid.

Initiating device auth flow to setup offline access token.

Decoding response

Please authorise using the following endpoint.

Verification URL: https://auth.dev.rrap-is.com/auth/realms/rrap/device?user_code=RWGA-ZEOE
User Code: RWGA-ZEOE

Awaiting completion


Token generation complete. Authorisation successful.



### Authenticated call

In [4]:
# Pull out the BearerAuth function (don't call it yet)
auth = token_manager.get_auth

# Setup postfix and endpoint
postfix = "/registry/items/list-all-datasets"
endpoint = data_api + postfix

# When making the request, use auth=auth() - this will ensure tokens are valid
# right at the point of using them, including potentially auto refreshing!
response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json(), indent=4, sort_keys=True))

{
    "num_items": 85,
    "registry_items": [
        {
            "collection_format": {
                "author": {
                    "email": "rosspetridis@gmail.com",
                    "name": "Ross Petridis",
                    "orcid": "https://orcid.org/0000-0002-7190-5187",
                    "organisation": {
                        "name": "University of Melbourne",
                        "ror": "https://ror.org/00rqy9422"
                    }
                },
                "dataset_info": {
                    "created_date": "2021-12-25",
                    "description": "Results of counterfactual model run - updated",
                    "keywords": [
                        "keyword1",
                        "keyword2",
                        "coralLover"
                    ],
                    "license": "https://creativecommons.org/licenses/by/4.0/",
                    "name": "Coral results 2021-02-30 - updated",
                    "published_dat

## Test your auth against a endpoint 
Endpoint documentation can be found by appending either `/docs` or `/redoc` on the end a base URL.

For example:
- https://registry-api.testing.rrap-is.com/redoc
or
- https://registry-api.testing.rrap-is.com/docs

Then select from the menu an endpoint function call e.g. `/check-access/check-general-access`

Then append the function call onto the base url e.g. `https://registry-api.testing.rrap-is.com/check-access/check-general-access`

### Endpoint documentation redoc

In [5]:
IFrame("https://registry-api.testing.rrap-is.com/redoc", width=1024, height=600)

## Testing General Access Endpoint

In [6]:
#test general access endpoint
auth = token_manager.get_auth
base_url = "https://registry-api.testing.rrap-is.com"
url_query_string = "/check-access/check-general-access"
endpoint = base_url + url_query_string

response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json(), indent=4, sort_keys=True))

{
    "access_token": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIzMVBQVkZZdE53a2JEcVZnNjFqX08xNXdiN2FRZU1BaExCbWdpY2VvY2dBIn0.eyJleHAiOjE2NTg3Mjc4OTMsImlhdCI6MTY1ODcyNzU5MywiYXV0aF90aW1lIjoxNjU4NzI3NTkyLCJqdGkiOiI4MGU3MjhkYS1hODhmLTQ0Y2MtYjA4Yi1iNDBlZDk3MGMyMzkiLCJpc3MiOiJodHRwczovL2F1dGguZGV2LnJyYXAtaXMuY29tL2F1dGgvcmVhbG1zL3JyYXAiLCJhdWQiOlsicmVhbG0tbWFuYWdlbWVudCIsImFjY291bnQiXSwic3ViIjoiNDY4Y2E5MWMtZmM1My00YWJiLWJmNjYtNjExMzU3ZDQ3YzdmIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiY2xpZW50LXRvb2xzIiwic2Vzc2lvbl9zdGF0ZSI6Ijg1OGVjY2FiLWI4NDgtNDY3ZS05ZTdkLTBiNTIxZDVmZDNiOSIsImFjciI6IjEiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZW50aXR5LXJlZ2lzdHJ5LWFkbWluIiwiZGF0YS1zdG9yZS13cml0ZSIsInN5cy1hZG1pbi1yZWFkIiwicHJvdi1hZG1pbiIsInN5cy1hZG1pbi13cml0ZSIsInByb3YtcmVhZCIsImVudGl0eS1yZWdpc3RyeS13cml0ZSIsImRlZmF1bHQtcm9sZXMtcnJhcCIsInByb3Ytd3JpdGUiLCJlbnRpdHktcmVnaXN0cnktcmVhZCIsIm9mZmxpbmVfYWNjZXNzIiwiZGF0YS1zdG9yZS1hZG1pbiIsImRhdGEtc3RvcmUtcmVhZCIsInVtYV9hdXRob3JpemF0aW9uIiwic3lzLWFkbWluLWFkbWluIl19LCJyZXNvd

In [7]:
#test read access endpoint
auth = token_manager.get_auth
base_url = "https://registry-api.testing.rrap-is.com"
url_query_string = "/check-access/check-read-access"
endpoint = base_url + url_query_string

response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json(), indent=4, sort_keys=True))

{
    "access_token": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIzMVBQVkZZdE53a2JEcVZnNjFqX08xNXdiN2FRZU1BaExCbWdpY2VvY2dBIn0.eyJleHAiOjE2NTg3Mjc4OTMsImlhdCI6MTY1ODcyNzU5MywiYXV0aF90aW1lIjoxNjU4NzI3NTkyLCJqdGkiOiI4MGU3MjhkYS1hODhmLTQ0Y2MtYjA4Yi1iNDBlZDk3MGMyMzkiLCJpc3MiOiJodHRwczovL2F1dGguZGV2LnJyYXAtaXMuY29tL2F1dGgvcmVhbG1zL3JyYXAiLCJhdWQiOlsicmVhbG0tbWFuYWdlbWVudCIsImFjY291bnQiXSwic3ViIjoiNDY4Y2E5MWMtZmM1My00YWJiLWJmNjYtNjExMzU3ZDQ3YzdmIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiY2xpZW50LXRvb2xzIiwic2Vzc2lvbl9zdGF0ZSI6Ijg1OGVjY2FiLWI4NDgtNDY3ZS05ZTdkLTBiNTIxZDVmZDNiOSIsImFjciI6IjEiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZW50aXR5LXJlZ2lzdHJ5LWFkbWluIiwiZGF0YS1zdG9yZS13cml0ZSIsInN5cy1hZG1pbi1yZWFkIiwicHJvdi1hZG1pbiIsInN5cy1hZG1pbi13cml0ZSIsInByb3YtcmVhZCIsImVudGl0eS1yZWdpc3RyeS13cml0ZSIsImRlZmF1bHQtcm9sZXMtcnJhcCIsInByb3Ytd3JpdGUiLCJlbnRpdHktcmVnaXN0cnktcmVhZCIsIm9mZmxpbmVfYWNjZXNzIiwiZGF0YS1zdG9yZS1hZG1pbiIsImRhdGEtc3RvcmUtcmVhZCIsInVtYV9hdXRob3JpemF0aW9uIiwic3lzLWFkbWluLWFkbWluIl19LCJyZXNvd

In [8]:
#test write access endpoint
auth = token_manager.get_auth
base_url = "https://registry-api.testing.rrap-is.com"
url_query_string = "/check-access/check-write-access"
endpoint = base_url + url_query_string

response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json(), indent=4, sort_keys=True))

{
    "access_token": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIzMVBQVkZZdE53a2JEcVZnNjFqX08xNXdiN2FRZU1BaExCbWdpY2VvY2dBIn0.eyJleHAiOjE2NTg3Mjc4OTMsImlhdCI6MTY1ODcyNzU5MywiYXV0aF90aW1lIjoxNjU4NzI3NTkyLCJqdGkiOiI4MGU3MjhkYS1hODhmLTQ0Y2MtYjA4Yi1iNDBlZDk3MGMyMzkiLCJpc3MiOiJodHRwczovL2F1dGguZGV2LnJyYXAtaXMuY29tL2F1dGgvcmVhbG1zL3JyYXAiLCJhdWQiOlsicmVhbG0tbWFuYWdlbWVudCIsImFjY291bnQiXSwic3ViIjoiNDY4Y2E5MWMtZmM1My00YWJiLWJmNjYtNjExMzU3ZDQ3YzdmIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiY2xpZW50LXRvb2xzIiwic2Vzc2lvbl9zdGF0ZSI6Ijg1OGVjY2FiLWI4NDgtNDY3ZS05ZTdkLTBiNTIxZDVmZDNiOSIsImFjciI6IjEiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZW50aXR5LXJlZ2lzdHJ5LWFkbWluIiwiZGF0YS1zdG9yZS13cml0ZSIsInN5cy1hZG1pbi1yZWFkIiwicHJvdi1hZG1pbiIsInN5cy1hZG1pbi13cml0ZSIsInByb3YtcmVhZCIsImVudGl0eS1yZWdpc3RyeS13cml0ZSIsImRlZmF1bHQtcm9sZXMtcnJhcCIsInByb3Ytd3JpdGUiLCJlbnRpdHktcmVnaXN0cnktcmVhZCIsIm9mZmxpbmVfYWNjZXNzIiwiZGF0YS1zdG9yZS1hZG1pbiIsImRhdGEtc3RvcmUtcmVhZCIsInVtYV9hdXRob3JpemF0aW9uIiwic3lzLWFkbWluLWFkbWluIl19LCJyZXNvd

In [9]:
#test admin access endpoint
auth = token_manager.get_auth
base_url = "https://registry-api.testing.rrap-is.com"
url_query_string = "/check-access/check-admin-access"
endpoint = base_url + url_query_string

response = requests.get(endpoint, auth=auth())

print(json.dumps(response.json(), indent=4, sort_keys=True))

{
    "access_token": "eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICIzMVBQVkZZdE53a2JEcVZnNjFqX08xNXdiN2FRZU1BaExCbWdpY2VvY2dBIn0.eyJleHAiOjE2NTg3Mjc4OTMsImlhdCI6MTY1ODcyNzU5MywiYXV0aF90aW1lIjoxNjU4NzI3NTkyLCJqdGkiOiI4MGU3MjhkYS1hODhmLTQ0Y2MtYjA4Yi1iNDBlZDk3MGMyMzkiLCJpc3MiOiJodHRwczovL2F1dGguZGV2LnJyYXAtaXMuY29tL2F1dGgvcmVhbG1zL3JyYXAiLCJhdWQiOlsicmVhbG0tbWFuYWdlbWVudCIsImFjY291bnQiXSwic3ViIjoiNDY4Y2E5MWMtZmM1My00YWJiLWJmNjYtNjExMzU3ZDQ3YzdmIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiY2xpZW50LXRvb2xzIiwic2Vzc2lvbl9zdGF0ZSI6Ijg1OGVjY2FiLWI4NDgtNDY3ZS05ZTdkLTBiNTIxZDVmZDNiOSIsImFjciI6IjEiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZW50aXR5LXJlZ2lzdHJ5LWFkbWluIiwiZGF0YS1zdG9yZS13cml0ZSIsInN5cy1hZG1pbi1yZWFkIiwicHJvdi1hZG1pbiIsInN5cy1hZG1pbi13cml0ZSIsInByb3YtcmVhZCIsImVudGl0eS1yZWdpc3RyeS13cml0ZSIsImRlZmF1bHQtcm9sZXMtcnJhcCIsInByb3Ytd3JpdGUiLCJlbnRpdHktcmVnaXN0cnktcmVhZCIsIm9mZmxpbmVfYWNjZXNzIiwiZGF0YS1zdG9yZS1hZG1pbiIsImRhdGEtc3RvcmUtcmVhZCIsInVtYV9hdXRob3JpemF0aW9uIiwic3lzLWFkbWluLWFkbWluIl19LCJyZXNvd